# Practical training - Variational Method
## Performing the assimilation

the data assimilation is done through the minimization of the cost function:
$$
J(u^0) = J_b(u^0) + J_o(u^0) = \frac{1}{2}\, (u^0-u^b)^TB^{-1}(u^0-u^b) + \frac{1}{2}\, \sum_{k=0}^N  (H_k u^k-u^k_{obs})^TR^{-1}(H_k u^k-u^k_{obs})
$$
 
with
 
$$
B(x_i, x_j) = \sigma_B^2 \exp\left(-\frac{(x_i-x_j)^2}{2L_B^2}\right)
$$
 
or
 
$$
B_{i,j} = \sigma_B^2 \exp\left(-\frac{((i-j)\Delta x)^2}{2L_B^2}\right)
$$
 
and
 
$$ R=\sigma_R^2 Id $$


### Initialisation

In [ ]:
from burgers import *
from gausscov import *
from simvar import *
from obsopt import *
from plots import *

import numpy as np
import scipy.optimize as opt
import math




In [ ]:
# Space-time domain
nx = 40                     # number of grid points
dx = 1./nx                  # space step
xx = np.array(range(nx))*dx # grid points abscissa
dt = 0.5*dx                 # time step
nt = 20                     # number of time steps
ns = 0                      # numerical scheme

M=Burgers(nx,dx,dt,ns)



In [ ]:
# Error staristics
sigmab = 0.02              # background state error std
sigmao = 0.001             # Observation error std
Lb = 0.05                  # Correlation length for B matrix

# Assimilation Parameters

precond = True             # preconditioning by square root of B (1=yes)
iobstsub = 5                # Frequency of temporal subsampling of observations, [1:nt], 1=every time step
iobsxsub = 8                # Frequency of spatial subsampling of observations, [1:nx], 1=every space step

# Observation operator and error covariance matrix

H = Obsopt(nx,iobsxsub,nt,iobstsub)
R = sigmao*sigmao*np.eye(H.nobs,H.nobs)


In [ ]:
# Initialization of true field uo
uo=np.sin(2*math.pi*xx);
true=H.gen_obs(M,uo,sigmao)

# Initialization of background
ub=np.cos(2.*math.pi*xx)
ubkg=[ub]
for it in range(nt):
    ub=M.step(ub)
    ubkg.append(ub)


### Creation of variational objects and actual minimisation

In [ ]:
# Initialization of B matrix and its inverse

if precond:
    indic=2
else:
    indic=1
    
B=gausscov(nx,sigmab,Lb,indic)

# Actual minimisation

var=Variational(ubkg[0],nt,B,M,H,R,precond)


In [ ]:
uopt=np.zeros(nx)

Jini=var.cost(uopt)
costj=[Jini]

def callback(uu):
    costj.append(var.cost(uu))

res = opt.minimize(var.cost,uopt,
                   method='L-BFGS-B',
                   jac=var.grad,
                   options={'disp': True, 'gtol': 1e-05, 'maxiter': 10000, 'iprint':100},
                   callback=callback)
print()
print ('In the minimization successful? {}'.format(res.success))
print()
print ('Initial cost function value: {}'.format(Jini))
print()
print ('Final cost function value: {}'.format(res.fun))
print()
print ('Number of iterations: {}'.format(res.nit))
print()
print ('Final control vector: {}'.format(res.x))
print()

### Back transform the control vector if needed

In [ ]:
if precond:
    ua=ubkg[0] + B.sqr.dot(res['x'])
else:
    ua=ubkg[0] + res['x']

uana=[ua]
for it in range(nt):
    ua=M.step(ua)
    uana.append(ua)


### Plotting

In [ ]:
# plot

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from plots import animator

plt.rcParams["animation.html"] = "jshtml"
fig, ax = plt.subplots()

print('black: truth           blue: background          red: analysis')
anim=animator(xx,ax=ax,trajectories=[true,ubkg,uana],legends=['True','Background','Analysis'])
FuncAnimation(fig, anim.update, np.arange(nt),
                    init_func=anim.init)

In [ ]:
%matplotlib notebook
from IPython.display import clear_output, display, HTML

import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

plt.semilogy(range(len(costj)),costj)
plt.xlabel('Iterations')
plt.ylabel('J')
plt.show()

In [ ]:


f, axarr = plt.subplots(2, 2)

axarr[0, 0].plot(xx,true[0],'k-')
axarr[0, 0].plot(xx,ubkg[0],'b-')
axarr[0, 0].plot(xx,uana[0],'r-',linewidth=3)
axarr[0, 0].plot(np.dot(H.mat,xx),H.yo[0],'kd')
axarr[0, 0].legend(['True','Background','Analysis','Observations'])
axarr[0, 0].set_title('States at the begining of experiments')

axarr[0, 1].plot(xx,true[nt],'k-')
axarr[0, 1].plot(xx,ubkg[nt],'b-')
axarr[0, 1].plot(xx,uana[nt],'r-',linewidth=3)
axarr[0, 1].plot(np.dot(H.mat,xx),H.yo[nt],'kd')
axarr[0, 1].legend(['True','Background','Analysis','Observations'])
axarr[0, 1].set_title('States at the end of experiments')

axarr[1, 0].set_title('Errors at the begining of experiments')
axarr[1, 0].plot(xx,(ubkg[0]-true[0])**2,'b-',linewidth=3)
axarr[1, 0].plot(xx,(uana[0]-true[0])**2,'r-',linewidth=3)
axarr[1, 0].legend(['Squared background error','Squared analysis error'])
    
axarr[1, 1].set_title('Errors at the end of experiments')
axarr[1, 1].plot(xx,(ubkg[nt]-true[nt])**2,'b-',linewidth=3)
axarr[1, 1].plot(xx,(uana[nt]-true[nt])**2,'r-',linewidth=3)
axarr[1, 1].legend(['Squared background error','Squared analysis error'])
    
plt.show()